In [57]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
from itertools import permutations
from scipy.spatial import distance

In [58]:
wines = ['Class', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols',
              'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315',\
              'Proline']

In [59]:
f=pd.read_csv('C:\\Users\\Daniele\\Desktop\\HW 4\\wine.data',names=wines)

In [60]:
pd.DataFrame.drop(f,['Class'],axis=1,inplace=True)

In [61]:
f.head()

,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315,Proline
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [62]:
scaler = StandardScaler()

In [63]:
scaled_array = scaler.fit_transform(f)

In [64]:
scaled_data= pd.DataFrame( scaled_array, columns = f.columns )

In [65]:
scaled_data.head()

,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315,Proline
0,1.518613,-0.562250,0.232053,-1.169593,1.913905,0.808997,1.034819,-0.659563,1.224884,0.251717,0.362177,1.847920,1.013009
1,0.246290,-0.499413,-0.827996,-2.490847,0.018145,0.568648,0.733629,-0.820719,-0.544721,-0.293321,0.406051,1.113449,0.965242
2,0.196879,0.021231,1.109334,-0.268738,0.088358,0.808997,1.215533,-0.498407,2.135968,0.269020,0.318304,0.788587,1.395148
3,1.691550,-0.346811,0.487926,-0.809251,0.930918,2.491446,1.466525,-0.981875,1.032155,1.186068,-0.427544,1.184071,2.334574
4,0.295700,0.227694,1.840403,0.451946,1.281985,0.808997,0.663351,0.226796,0.401404,-0.319276,0.362177,0.449601,-0.037874


In [66]:
def Kmeans(array,k):
    
    def initialize_centroids(array,k):
        initial_centroids = np.random.permutation(array.shape[0])[:k]
        centroids = array[initial_centroids]
        return centroids
    
    lis_initial_centroids = initialize_centroids(array,k)
    
    def assign_centroids(array,centroids):
        lis_clustering=[]
        for e in array:
            d_e = []
            for i in range(k):
                d = distance.euclidean(e,centroids[i])
                d_e.append(d)
            min_distance=min(d_e)
            lis_clustering.append((d_e.index(min_distance)+1))
        lis_clustering= np.array(lis_clustering)
        clusters_to_append = lis_clustering.reshape(178,1)   
        new_array = np.append(array,clusters_to_append,axis=1)
        return new_array
    
    new_array = assign_centroids(scaled_array,lis_initial_centroids)
    
    def new_centroids(new_array):
        lis_centroids = []
        centroids = []
        for i in range(k):
            lis_centroids.append([])
        for j in range(178):
            for i in range(k):
                if float(i+1) == new_array[j][13]:
                    lis_centroids[i].append(new_array[j])
        for i in range(k):
            lis_centroid = lis_centroids[i]
            lis_centroid = np.delete(lis_centroid,13,axis=1)
            new_centroid = np.mean(lis_centroid,axis=0)
            centroids.append(new_centroid)
        return centroids
   
    centroids = new_centroids(new_array)
    
    final_array = assign_centroids(scaled_array,centroids)
    
    def update_centroids(initial_centroids):
        centroids_old = np.zeros((k,12))
        centroids = initial_centroids
        numItem = 0
        while ([list(elem) for elem in centroids_old] != [list(elem) for elem in centroids]):
            numItem += 1
            centroids_old = centroids.copy()
            new_array = assign_centroids(scaled_array,centroids_old)
            centroids = new_centroids(new_array)
        return numItem,centroids,centroids_old
    
    numItem = update_centroids(lis_initial_centroids)[0]
    
    centroids = update_centroids(lis_initial_centroids)[1]
    
    final_array = assign_centroids(scaled_array,centroids)
    
    def clustering(array):
        lis_clusters = []
        clusters = []
        for i in range(k):
            lis_clusters.append([])
        for j in range(178):
            for i in range(k):
                if float(i+1) == array[j][13]:
                    lis_clusters[i].append(array[j])
        for i in range(k):
            cluster = lis_clusters[i]
            cluster = np.delete(cluster,13,axis=1)
            clusters.append(cluster)
        return clusters
    
    clusters = clustering(final_array)
    
    def dissimilarity(clusters,centroids):
        Dist = []
        for c in range (len(clusters)):
            Dist_cluster = ([])
            for e in clusters[c]:
                Dist_cluster += [(distance.euclidean(e,centroids[c]))**2]
            Dist += [Dist_cluster]
        final_dist = []
        for e in Dist:
            e = (sum(e))
            final_dist += [e]
        totDist = (sum(final_dist))
        return totDist
    
    totDist = dissimilarity(clusters,centroids)
    
    return totDist,clusters,centroids,lis_initial_centroids

In [67]:
results = Kmeans(scaled_array,3)

In [68]:
results

(1277.9284888446423,
 [array([[-7.78980294e-01, -1.25345042e+00, -3.67916223e+00,
          -2.67101814e+00, -8.24415067e-01, -5.04914475e-01,
          -1.46505818e+00, -6.59563114e-01, -2.05151334e+00,
          -1.34446639e+00,  4.06050663e-01, -1.11821035e+00,
          -7.22540161e-01],
         [-8.28390893e-01, -1.10982432e+00, -3.16248596e-01,
          -1.04947918e+00,  8.83583612e-02, -3.92751267e-01,
          -9.42995485e-01,  2.16066901e+00, -2.06903418e+00,
          -7.73474010e-01,  1.28351841e+00, -1.33007677e+00,
          -2.13021163e-01],
         [ 8.26864161e-01, -9.75174845e-01, -1.63217132e+00,
          -4.48909194e-01, -4.03135023e-01, -3.12634689e-01,
          -2.40218779e-01, -3.37250872e-01, -1.50836726e+00,
          -5.44211919e-01,  1.19577163e+00, -2.14246944e-01,
          -3.72245850e-01],
         [-7.78980294e-01, -1.08289442e+00, -7.54889505e-01,
          -1.48624201e-01, -8.94628408e-01,  1.93062948e+00,
           1.07497763e+00, -1.38476566e+0

In [69]:
def bestKmeans(rep):
    best = Kmeans(scaled_array,3)
    minDissimilarity = best[0]
    for trial in range(rep):
        result = Kmeans(scaled_array,3)
        currDissimilarity = result[0]
        if currDissimilarity < minDissimilarity:
            best = result
            minDissimilarity = currDissimilarity       
    return best[0],best[3]
    
    

In [70]:
def worstKmeans(rep):
    worst = Kmeans(scaled_array,3)
    minDissimilarity = worst[0]
    for trial in range(rep):
        result = Kmeans(scaled_array,3)
        currDissimilarity = result[0]
        if currDissimilarity > minDissimilarity:
            worst = result
            minDissimilarity = currDissimilarity       
    return worst[0],worst[3]

In [71]:
best = bestKmeans(50)

In [72]:
worst = worstKmeans(50)

In [73]:
best

(1277.9284888446423,
 array([[ 1.38273339, -0.76871232, -0.17003496, -0.80925118, -0.33292168,
         -0.15240153,  0.40231992, -0.82071924, -0.03661659, -0.02512794,
          0.93253131,  0.29423247,  1.69767508],
        [ 1.02450655, -0.68792264,  0.92656731,  0.15166079,  1.07134513,
          1.04934713,  1.37616764,  0.30737361,  0.22619603,  0.66698403,
          0.75703776, -0.05887823,  1.22000102],
        [-0.37134286,  1.3767026 ,  0.12239231,  1.05251577,  0.08835836,
          0.85706734,  0.52279593,  0.5491078 ,  0.62917538, -1.076273  ,
          1.02027808,  0.73208974, -0.9040563 ]]))

In [74]:
worst

(1601.5261762660718,
 array([[ 1.71625494, -0.4186237 ,  0.30515936, -1.46987817, -0.26270834,
          0.32829793,  0.49267693, -0.49840699,  0.6817379 ,  0.08301456,
          0.2744305 ,  1.36768901,  1.72952002],
        [ 1.65449169, -0.58917969,  1.21899459,  1.65308575, -0.12228166,
          0.80899739, -0.72212281,  1.3548884 ,  1.94323848,  3.43543192,
         -1.69987192, -0.92046835, -0.27671104],
        [ 0.49334262,  1.41260912,  0.41481959,  1.05251577,  0.1585717 ,
         -0.79333415, -1.28434417,  0.5491078 , -0.31695005,  0.96978302,
         -1.12951789, -1.48544548,  0.0098934 ]]))